In [1]:
import os
import sys

spark_home = '/opt/spark'
os.environ['SPARK_HOME'] = spark_home

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.1-src.zip'))

execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.10 (default, Oct 23 2015 18:05:06)
SparkContext available as sc, HiveContext available as sqlContext.


Check [http://localhost:4040/](http://localhost:4040)

In [55]:
birdstrikes = sc.textFile("birdstrikes.csv")

In [56]:
birdstrikes.first()

u'id,aircraft,flight_date,damage,airline,state,phase_of_flight,reported_date,bird_size,cost,feet_above_ground'

In [57]:
birdstrikes.take(2)

[u'id,aircraft,flight_date,damage,airline,state,phase_of_flight,reported_date,bird_size,cost,feet_above_ground',
 u'2,Airplane,2000-01-01,No damage,CONTINENTAL AIRLINES,New Jersey,Take-off run,,Medium,0,0']

In [58]:
birdstrikes.count()

10000

In [24]:
fromlocal = sc.parallelize(["hello","ceu","tonight"])

In [27]:
fromlocal.take(2)

['hello', 'ceu']

In [50]:
fromlocal.collect()

['hello', 'ceu', 'tonight']

**Partitioning**

In [28]:
birdstrikes.getNumPartitions()

2

In [29]:
birdstrikes.repartition(20)

MapPartitionsRDD[31] at repartition at NativeMethodAccessorImpl.java:-2

In [30]:
birdstrikes.getNumPartitions()

2

Uh oh. (Immutability)

In [31]:
rdd = birdstrikes.repartition(10)

In [51]:
rdd.getNumPartitions()

10

In [52]:
birdstrikes.first()

u'id,aircraft,flight_date,damage,airline,state,phase_of_flight,reported_date,bird_size,cost,feet_above_ground'

In [53]:
rdd.first()

u'11,Airplane,2000-01-02,No damage,UNITED AIRLINES,California,Take-off run,,Small,0,0'

**lambda functions**

In [54]:
def is_not_header(row):
    if row.startswith("id,"):
        return False
    else:
        return True


In [38]:
rows = birdstrikes.filter(is_not_header)

In [39]:
rows.take(5)

[u'2,Airplane,2000-01-01,No damage,CONTINENTAL AIRLINES,New Jersey,Take-off run,,Medium,0,0',
 u'3,Airplane,2000-01-01,No damage,UNITED AIRLINES,,,,Medium,0,',
 u'4,Airplane,2000-01-01,No damage,UNITED AIRLINES,Colorado,Climb,,Medium,0,',
 u'5,Airplane,2000-01-01,No damage,UNITED AIRLINES,Illinois,Approach,,Medium,0,',
 u'7,Airplane,2000-01-01,Caused damage,US CUSTOMS AND BORDER PROTECTION,Florida,En Route,,Large,0,1000']

In [43]:
def is_not_header(row):
    return not row.startswith("id,")

In [44]:
rows = birdstrikes.filter(is_not_header)

In [45]:
rows.take(5)

[u'2,Airplane,2000-01-01,No damage,CONTINENTAL AIRLINES,New Jersey,Take-off run,,Medium,0,0',
 u'3,Airplane,2000-01-01,No damage,UNITED AIRLINES,,,,Medium,0,',
 u'4,Airplane,2000-01-01,No damage,UNITED AIRLINES,Colorado,Climb,,Medium,0,',
 u'5,Airplane,2000-01-01,No damage,UNITED AIRLINES,Illinois,Approach,,Medium,0,',
 u'7,Airplane,2000-01-01,Caused damage,US CUSTOMS AND BORDER PROTECTION,Florida,En Route,,Large,0,1000']

In [59]:
rows = birdstrikes.filter(lambda row: not row.startswith("id,"))

In [62]:
rows.take(5)

[u'2,Airplane,2000-01-01,No damage,CONTINENTAL AIRLINES,New Jersey,Take-off run,,Medium,0,0',
 u'3,Airplane,2000-01-01,No damage,UNITED AIRLINES,,,,Medium,0,',
 u'4,Airplane,2000-01-01,No damage,UNITED AIRLINES,Colorado,Climb,,Medium,0,',
 u'5,Airplane,2000-01-01,No damage,UNITED AIRLINES,Illinois,Approach,,Medium,0,',
 u'7,Airplane,2000-01-01,Caused damage,US CUSTOMS AND BORDER PROTECTION,Florida,En Route,,Large,0,1000']

In [63]:
rows.count()

9999

In [64]:
hello = rows.map(lambda row: "hello," + row)

In [65]:
hello.take(5)

[u'hello,2,Airplane,2000-01-01,No damage,CONTINENTAL AIRLINES,New Jersey,Take-off run,,Medium,0,0',
 u'hello,3,Airplane,2000-01-01,No damage,UNITED AIRLINES,,,,Medium,0,',
 u'hello,4,Airplane,2000-01-01,No damage,UNITED AIRLINES,Colorado,Climb,,Medium,0,',
 u'hello,5,Airplane,2000-01-01,No damage,UNITED AIRLINES,Illinois,Approach,,Medium,0,',
 u'hello,7,Airplane,2000-01-01,Caused damage,US CUSTOMS AND BORDER PROTECTION,Florida,En Route,,Large,0,1000']

In [69]:
x = "2,Airplane,2000-01-01,No damage,CONTINENTAL AIRLINES,New Jersey,Take-off run,,Medium,0,0"
x.split(",")

['2',
 'Airplane',
 '2000-01-01',
 'No damage',
 'CONTINENTAL AIRLINES',
 'New Jersey',
 'Take-off run',
 '',
 'Medium',
 '0',
 '0']

In [ ]:
lists = rows.map(lambda row: row.split(","))

In [86]:
states = lists.map(lambda item: [ item[5], int(item[9]) ])

In [87]:
states.filter(lambda item: item[0] == 'Ohio').take(5)

[[u'Ohio', 0], [u'Ohio', 0], [u'Ohio', 0], [u'Ohio', 0], [u'Ohio', 0]]

In [89]:
df = states.toDF(["state","cost"])

In [94]:
df.toPandas().head()

,state,cost
0,New Jersey,0
1,,0
2,Colorado,0
3,Illinois,0
4,Florida,0


In [95]:
df.registerTempTable("incidents")

In [97]:
sqlContext.sql("SELECT * FROM incidents LIMIT 10").toPandas()

,state,cost
0,New Jersey,0
1,,0
2,Colorado,0
3,Illinois,0
4,Florida,0
5,,0
6,Ohio,0
7,Florida,0
8,California,0
9,Utah,0


In [100]:
sqlContext.sql("SELECT state, SUM(cost) as total_costs FROM incidents GROUP BY state LIMIT 10").toPandas()

,state,total_costs
0,Idaho,12925
1,Michigan,1334991
2,Texas,1079974
3,Virgin Islands,0
4,West Virginia,565892
5,Alberta,0
6,Louisiana,1607910
7,Quebec,0
8,Manitoba,0
9,Puerto Rico,18222
